In [51]:
import math

#HEAT EXCHANGER EQUATIONS (GYROID)
def Dh(epsi,ssa): #Define the function that returns the value of hydraulic diameter (Reynolds et. al)
    ''' 
    epsi = porosity (fluid volume (Vf) / total volume (Vt))
    ssa = specific surface area (Wetted area for Fluid 1,2 or 3 / )
    '''
    Dh1 = (4*epsi)/ssa 
    return Dh1 

def Reynolds(rho,V,Dh,mu):
    '''
    rho = fluid density
    V = fluid velocity (can be obtained with m' = rho*V*A)
    Dh = Hydraulic diameter (can be obtained with the expression from Reynolds et. al)
    mu = dynamic viscosity
    '''
    Reh = (rho*V*Dh)/mu
    return Reh

def Dtlmtd(tc_in, th_in, tc_out, th_out):
    '''
    tc = temp.cold
    th = temp.hot
    '''
    Dt_lmtd = ( (tc_in - th_out) - (tc_out - th_in) ) / math.log(((tc_in - th_out))/(tc_out - th_in))
    return Dt_lmtd

def htcoef(Q, A, Dt_lmtd):
    '''
    Q = Heat transfer Rate
    A = Effective heat transfer area 
    Dt_lmtd = log mean temp difference
    '''
    U = Q/(A*Dt_lmtd)
    return U

def j(Nu, Re, Pr):
    '''
    j = Colburn Factor
    Nu = Nusselt 
    Re = Reynolds
    Pr = Prandt
    '''
    j = Nu / (Re * (Pr**(1/3)))
    return j

def f(dP, Dh, rho, u, L):
    '''
    dP = pressure drop
    Dh = Hydraulic diameter
    rho = fluid density
    u = fluid velocity
    L = channel length
    '''
    f = (dP * Dh) / (2*rho*(u**2)*L)
    return f




In [52]:
# EPSI-NUT METHOD (ASSUMING THE FLOW IS TURBULENT FLOW IN THE TPMS STRUCTURE)
def NusseltCor(Re,Pr): # REYNOLDS ET AL. NUSSELT CORRELATION FOR GYROID, 100 < RE < 2500
    '''
    Author does not provide the cell size value
    Channel porosity: 39.2, Channel Dh = 4.3 mm, total porosity: 78.3
    Gyroid
    '''
    Nu = 0.49 * (math.pow(Re, 0.62)) * (math.pow(Pr, 0.4))
    return Nu

def NTU(U,A,Cmin):
    NTU = (U*A)/Cmin

def epsilon(Cr, NTU):
    epsi = 1 - math.exp((1/Cr)*(math.pow(NTU,0.22)*(math.exp(-Cr*(math.pow(NTU,0.78))-1))))


In [53]:
# PRESSURE DROP 
import numpy as np
import math

def Dh(vf, afs): #OBTIDO NO ARTIGO QUE PROF KLEBER ENVIOU (NÃO AUTORIZADO COMPARTILHAMENTO)
    '''
    vf = representa o volume de fluído
    afs = área de interface solido/fluido ( verificar a definição no artigo nao publicado, ambas equações são iguais)
    '''
    Dh = (4*vf)/afs
    return Dh

def Re(rho, V, Dh, mu):
    Re = (rho*V*Dh)/mu
    return Re

def fcorrelation1(Re): # kaixin Yan et al. 2024 correlation
    f = 2.577*(np.power(Re, -0.095)) 
    return f

def fcorrelation2(Re): #kaixin Yan et al #GYROID AS PRINTED
    f = 2.756*(np.power(Re, -0.166))  
    return f

def fcorrelation3(Re): # kaixin Yan et al #GYROID AS POLISHED 
    f = 2.344*(np.power(Re, -0.081)) 
    return f

def dP(f,L,epsi,Dh,rho,V): #REYNOLDS et al.
    dP = (f*L*2*rho*(np.power(V, 2))) / (np.power(epsi, 2)*Dh) 



In [54]:
#GEOMETRY FROM NTOPOLOGY
vthex = 195461.4765 #mm³ total volume of the hex core
vtf2 = 125578.5438 #mm³ total volume of the mid core
vtf3 = 83879.8214 #mm³ total volume of the outer core
vf1 = 44195.7598 #mm³ total volume of the fluid1
vf2 = 27184.1405 #mm³ total volume of the fluid2
vf3 = 14342.9852 #mm³ total volume of the fluid3
Lf1f2 = 100/1000 #m
Lf3 = 80/1000 #m


f1sa = 94009.046 #mm² f1 wetted area
f2sa = 50021.1927 #mm² f2 wetted area
f3sa = 36110.0301 #mm² f3 wetted area

epsif1 = vf1/vthex
epsif2 = vf2/vtf2
epsif3 = vf3/vtf3

In [61]:
import numpy as np
import CoolProp.CoolProp as CP
import math
import plotly.offline as pyo
import plotly.graph_objects as go

muair = CP.PropsSI('V', 'T', 300, 'P', 101325, 'Air')  # Viscosidade dinâmica
rhoair = CP.PropsSI('D', 'T', 300, 'P', 101325, 'Air')  # Densidade

# Propriedades da água
muw = CP.PropsSI('V', 'T', 300, 'P', 101325, 'Water')  # Viscosidade dinâmica
rhow = CP.PropsSI('D', 'T', 300, 'P', 101325, 'Water')  # Densidade

mar = np.linspace(0.05, 0.1, 100) #kg/s
mw = np.linspace(0.01, 0.1, 100) #kg/s

Dhf1 = Dh(vf1, f1sa)
Dhf2 = Dh(vf2, f2sa)
Dhf3 = Dh(vf3, f3sa)

Var = mar / (rhoair*Dhf1)
Vw = mw / (rhow*Dhf2)

Ref1 = Re(rhoair, Var, Dhf1, muair)
Ref2 = Re(rhow, Vw, Dhf2, muw)
Ref3 = Re(rhow, Vw, Dhf3, muw)

#F CORRELATION_1
f1air = fcorrelation1(Ref1)
f1w1 = fcorrelation1(Ref2)
f1w2 = fcorrelation1(Ref3)

#F CORRELATION_2
f2air = fcorrelation2(Ref1)
f2w1 = fcorrelation2(Ref2)
f2w2 = fcorrelation2(Ref3)

#F CORRELATION_1
f3air = fcorrelation3(Ref1)
f3w1 = fcorrelation3(Ref2)
f3w2 = fcorrelation3(Ref3)

# Queda de pressão
Dpf1 = dP(Lf1f2, epsif1, Dhf1, rhoair, Var, f1air)
Dpf2 = dP(Lf1f2, epsif2, Dhf2, rhow, Vw, f2air)
Dpf3 = dP(Lf3, epsif3, Dhf3, rhow, Vw, f3air)

# PLOT dP/Re
trace1 = go.Scatter(x=Ref1, y=Dpf1, mode='markers', name='f1', marker=dict(color='blue'))
trace2 = go.Scatter(x=Ref2, y=Dpf2, mode='markers', name='f2', marker=dict(color='red'))
trace3 = go.Scatter(x=Ref3, y=Dpf3, mode='markers', name='f3', marker=dict(color='green'))

layout = go.Layout(title='Dp/Re', xaxis=dict(title='Reynolds Number'), yaxis=dict(title='Pressure Drop (dP)'))

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
pyo.plot(fig)

print(f1air)



[1.21668113 1.21552002 1.21437156 1.21323548 1.21211153 1.21099946
 1.20989904 1.20881004 1.20773224 1.2066654  1.20560933 1.20456381
 1.20352865 1.20250366 1.20148863 1.20048339 1.19948776 1.19850157
 1.19752465 1.19655682 1.19559793 1.19464783 1.19370636 1.19277337
 1.19184871 1.19093225 1.19002385 1.18912337 1.18823069 1.18734566
 1.18646818 1.18559811 1.18473534 1.18387975 1.18303124 1.18218967
 1.18135496 1.18052699 1.17970566 1.17889087 1.17808252 1.17728051
 1.17648475 1.17569514 1.17491161 1.17413404 1.17336237 1.17259651
 1.17183636 1.17108186 1.17033292 1.16958946 1.16885142 1.1681187
 1.16739124 1.16666897 1.16595182 1.16523972 1.1645326  1.16383039
 1.16313303 1.16244046 1.16175262 1.16106943 1.16039085 1.15971681
 1.15904725 1.15838213 1.15772138 1.15706494 1.15641277 1.15576482
 1.15512102 1.15448133 1.1538457  1.15321409 1.15258643 1.15196269
 1.15134282 1.15072677 1.1501145  1.14950597 1.14890112 1.14829992
 1.14770233 1.14710831 1.1465178  1.14593079 1.14534722 1.14476